# Chapter 4 - Section 3 - Ascher and Greif - Second Edition

## Forward Substitution

In [ ]:
function forsub_org(A, b, p)
    n = length(b)
    y = zeros(n)
    
    # Permute b according to p
    b = b[p]
    
    # Forward substitution
    y[1] = b[1]
    for k = 2:n
        y[k] = (b[k] - A[k, 1:k-1]' * y[1:k-1])/A[k,k]
    end
    return y
end


forsub_org (generic function with 1 method)

In [2]:
L=[1 0 0;2 3 0;4 5 6]


3×3 Matrix{Int64}:
 1  0  0
 2  3  0
 4  5  6

In [3]:
forsub_org(L,ones(3),[1;2;3])

3-element Vector{Float64}:
  1.0
 -0.3333333333333333
 -0.22222222222222224

### Comparison of forward substituion with preprogrammed linear system solver in Julia

In [4]:
L\ones(3)

3-element Vector{Float64}:
  1.0
 -0.3333333333333333
 -0.22222222222222224

## Backward Substitution

In [5]:
function backsub(A, b)
    n = length(b)
    x = copy(b)
    
    # Backward substitution
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1
        x[k] = (b[k] - A[k, k+1:n]' * x[k+1:n]) / A[k, k]
    end
    
    return x
end


backsub (generic function with 1 method)

In [6]:
U=[1 2 3;0 4 5;0 0 6]

3×3 Matrix{Int64}:
 1  2  3
 0  4  5
 0  0  6

In [7]:
backsub(U,ones(3))

3-element Vector{Float64}:
 0.41666666666666663
 0.041666666666666685
 0.16666666666666666

### Comparison of solution of back-substitution with preprogrammed linear system solver in Julia

In [8]:
U\ones(3)

3-element Vector{Float64}:
 0.41666666666666663
 0.041666666666666685
 0.16666666666666666

## GEPP or PLU decompostion function

In [9]:
function plu(B)
    A=Float64.(copy(B))
    n = size(A, 1)
    
    # Initialize permutation vector p
    p = collect(1:n)
    
    # LU decomposition with partial pivoting
    for k = 1:n-1
        # Find row index of the maximum element in column k (for partial pivoting)
        _, q = findmax(abs.(A[k:n, k]))
        q = q + k - 1
        
        # Interchange rows k and q and record this in p
        A[[k; q], :] = A[[q; k], :]
        p[[k; q]] = p[[q; k]]
        
        # Compute the corresponding column of L
        J = k+1:n
        A[J, k] .= A[J, k] / A[k, k]
        
        # Update submatrix by outer product
        A[J, J] .-= A[J, k] * A[k, J]'
    end
    
    return p, A
end


plu (generic function with 1 method)

In [21]:
A=[1 0 3;0 1 6;7 1 9]

3×3 Matrix{Int64}:
 1  0  3
 0  1  6
 7  1  9

In [11]:
p,LUa=plu(A)

([3, 2, 1], [7.0 1.0 9.0; 0.0 1.0 6.0; 0.14285714285714285 -0.14285714285714285 2.5714285714285716])

In [12]:
p #vector of row-exchanges

3-element Vector{Int64}:
 3
 2
 1

In [13]:
LUa 

3×3 Matrix{Float64}:
 7.0        1.0       9.0
 0.0        1.0       6.0
 0.142857  -0.142857  2.57143

**Important:** LUa is the matrix with strict lower-triangular part (not including diagonal) = strict lower triangular part of unit lower triangular matrix L in LU decomposition and upper-triangular part (including diagonal) as the upper-triangular matrix U in LU decomposition

### Comparision with preprogrammed PLU decomposition in Julia

In [14]:
using LinearAlgebra
L,U,p=lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0        0.0       0.0
 0.0        1.0       0.0
 0.142857  -0.142857  1.0
U factor:
3×3 Matrix{Float64}:
 7.0  1.0  9.0
 0.0  1.0  6.0
 0.0  0.0  2.57143

In [15]:
L 

3×3 Matrix{Float64}:
 1.0        0.0       0.0
 0.0        1.0       0.0
 0.142857  -0.142857  1.0

In [16]:
U

3×3 Matrix{Float64}:
 7.0  1.0  9.0
 0.0  1.0  6.0
 0.0  0.0  2.57143

In [17]:
p

3-element Vector{Int64}:
 3
 2
 1

In [18]:
## or try this way
a=lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0        0.0       0.0
 0.0        1.0       0.0
 0.142857  -0.142857  1.0
U factor:
3×3 Matrix{Float64}:
 7.0  1.0  9.0
 0.0  1.0  6.0
 0.0  0.0  2.57143

In [19]:
a.L

3×3 Matrix{Float64}:
 1.0        0.0       0.0
 0.0        1.0       0.0
 0.142857  -0.142857  1.0

In [20]:
a.U

3×3 Matrix{Float64}:
 7.0  1.0  9.0
 0.0  1.0  6.0
 0.0  0.0  2.57143

In [21]:
a.p

3-element Vector{Int64}:
 3
 2
 1

In [22]:
a.P

3×3 Matrix{Float64}:
 0.0  0.0  1.0
 0.0  1.0  0.0
 1.0  0.0  0.0

## Our very own linear system solver that uses LU decomposition

In [23]:
function forsub(A, b, p) #forward substitution modified for unit lower triangular matrix
    n = length(b)
    y = zeros(n)
    
    # Permute b according to p
    b = b[p]
    
    # Forward substitution
    y[1] = b[1]
    for k = 2:n
        y[k] = (b[k] - A[k, 1:k-1]' * y[1:k-1])
    end
    
    return y
end


function ainvb(A, b)
    # Perform LU factorization with pivoting
    p,LUa = plu(A)
    
    # Forward substitution
    y = forsub(LUa,b,p)
    
    # Backward substitution
    x = backsub(LUa,y)

    return x
end

ainvb (generic function with 1 method)

In [24]:
ainvb(A,[1;0;-1])

3-element Vector{Float64}:
 -0.3333333333333333
 -2.666666666666666
  0.44444444444444436

### Comparison of ainvb with pre-programmed linear system solver in Julia

In [26]:
A\[1;0;-1]

3-element Vector{Float64}:
 -0.3333333333333333
 -2.666666666666666
  0.44444444444444436

In [53]:
# matrix solver
# Ax = b
using LinearAlgebra

function matrixSolver(A, b)
    n = size(A)[1]
    A = copy(A)  # Make copies and ensure Float64
    b = copy(b)
    L = Matrix{Float64}(I, n, n)  # Explicitly Float64 identity matrix
    P = collect(1:n)
    
    # Forward elimination
    for i = 1:n-1
        # Pivot
        _, k = findmax(abs.(A[i:n, i]))
        k = k + i - 1  # Adjust index
        
        # Swap rows
        A[k, :], A[i, :] = A[i, :], A[k, :]
        b[k], b[i] = b[i], b[k]
        P[k], P[i] = P[i], P[k]
        
        # Eliminate
        for r = i+1:n
            factor = A[r, i]/A[i, i]
            A[r, :] -= factor * A[i, :]
            L[r, i] = factor  # Now this will work with Float64
        end
    end

    # Forward substitution (Ly = b)
    y = zeros(Float64, n)  # Explicitly Float64
    for row = 1:n
        sum = 0.0  # Explicitly Float64
        for prev = 1:row-1
            sum += y[prev]*L[row,prev]
        end
        y[row] = b[row] - sum
    end

    # Backward substitution (Ux = y)
    x = zeros(Float64, n)  # Explicitly Float64
    for row = n:-1:1
        prev_sum = 0.0  # Explicitly Float64
        for prev = row+1:n
            prev_sum += x[prev]*A[row,prev]
        end
        x[row] = (y[row] - prev_sum) / A[row,row]
    end
    
    return x
end


matrixSolver (generic function with 1 method)

In [54]:
A=Float64[1 0 3;0 1 6;7 1 9]
matrixSolver(A,[1;0;-1])

3-element Vector{Float64}:
 -0.3333333333333333
 -2.666666666666666
  0.44444444444444436

In [ ]:
A\b

3-element Vector{Float64}:
 0.0
 0.0
 0.3333333333333333

In [2]:
for i = 1:0
    display(i)
end

In [9]:
for i = 10:-1:1
    print(i)
end

10987654321

In [32]:
using LinearAlgebra

function gaussian_elimination(U, L, p, b)
    n = size(U)[1]
    for column=1:n-1
        # pivot matrix
        _, k = findmax(U[column:n, column])
        # k is relative
        # row number to pivot is
        # n - column + 1 is the length of the column from row... in those any elements our row is 0 + column and that is column + k -1
        row_to_switch = column + k -1
        # swap
        U[column, :], U[row_to_switch, :] = U[row_to_switch, :], U[column, :]
        p[column], p[row_to_switch] = p[row_to_switch], p[column]
        b[column], b[row_to_switch] = b[row_to_switch], b[column]
        
        # now pivoting is done
        # now we do elimination of column
        for row=column+1:n
            factor = U[row, column]/U[column, column]
            U[row,column] = 0
            for sub=column+1:n
                U[row, sub] -= U[column, sub]*factor
            end
            L[row,column] = factor
        end
    end
end

function forward_elim(L, y, b)
    # Ly = pb
    n = size(L)[1]

    for row=1:n
        # we know y[1:row-1]
        # eqn is L[row, 1:row] * y[1:row] = b[row]
        # y[row] = b[row] - L[row, 1:row-1] * y[1:row-1]
        y[row] = b[row] - transpose(L[row, 1:row-1]) * y[1:row-1]
    end
end
    
function backward_elim(U, x, y)
    # Ux = y
    n = size(U)[1]
    for row=n:-1:1
        # we know x[row+1:n]
        x[row] = (y[row] - transpose(U[row, row+1:n])*x[row+1:n])/U[row,row]
    end
end

function matrixSolverAgain(A, b)
    n = size(A)[1]
    L = Matrix{Float64}(I, n, n)
    U = copy(A)
    p = collect(1:n)

    # let's form U and L using gaussian elimination
    gaussian_elimination(U, L, p, b)
    # now we have pA = LU
    # LUx = pb

    # now we have to do forward elimination
    # Ly = pb
    y = Float64.(zeros(n))
    forward_elim(L, y, b)

    x = Float64.(zeros(n))
    backward_elim(U, x, y)
    return x
end


matrixSolverAgain (generic function with 1 method)

In [33]:
A=Float64[1 0 3;0 1 6;7 1 9]
x = matrixSolverAgain(A,[1;0;-1])
display(A\[1;0;-1])
display(x)


3-element Vector{Float64}:
 -0.3333333333333333
 -2.666666666666666
  0.44444444444444436

3-element Vector{Float64}:
 -0.3333333333333333
 -2.666666666666666
  0.44444444444444436

In [ ]:
collect(1:)*Float64.([1;0;-1])[:]

-2.0